In [1]:
import time
import sys
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split, KFold
import multiprocessing
import os
from tqdm import tqdm

/Users/woongkook/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
 
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
 
    return False

In [5]:
weather_file = pd.read_csv("Data/weather.csv", sep=',')
weather_data = weather_file.drop('codesum', axis = 1) # codesum 컬럼 삭제
    
data = weather_data.values[:,2:] # index, station_nbr, date를 제외한 나머지 데이터 array로 변경
# 데이터가 있을 경우 float형으로 변경, 데이터가 없을 경우 unicodedata로 변경
for col in range(data.shape[1]):
    for row in range(data.shape[0]):
        if not is_number(data[row, col]):
            if row == 0:
                i = 1
                while not is_number(data[row+i, col]):
                    i += 1
                data[row, col] = data[row+i, col]
            else:
                data[row, col] = data[row-1, col]
        data[row, col] = float(data[row, col]) # Convert string to float value

weather_processed = np.append(weather_file.values[:,0:2], data, axis = 1)
df = pd.DataFrame(weather_processed, index=None, columns=weather_data.columns.values)
    
df

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,1,2012-01-01,52,31,42,16,36,40,23,0,716,1626,0,0.05,29.78,29.92,3.6,20,4.6
1,2,2012-01-01,48,33,41,16,37,39,24,0,716,1626,0,0.07,28.82,29.91,9.1,23,11.3
2,3,2012-01-01,55,34,45,9,24,36,20,0,735,1720,0,0,29.77,30.47,9.9,31,10
3,4,2012-01-01,63,47,55,4,28,43,10,0,728,1742,0,0,29.79,30.48,8,35,8.2
4,6,2012-01-01,63,34,49,0,31,43,16,0,727,1742,0,0,29.95,30.47,14,36,13.8
5,7,2012-01-01,50,33,42,0,26,35,23,0,727,1742,0,0,29.15,30.54,10.3,32,10.2
6,8,2012-01-01,66,45,42,0,34,46,23,0,727,1742,0,0,30.05,30.54,11,36,10.9
7,9,2012-01-01,34,19,27,0,17,23,38,0,727,1742,0,0,29.34,30.09,22.8,30,22.5
8,10,2012-01-01,73,53,63,0,55,58,2,0,723,1738,0,0,30.16,30.19,5.1,24,5.5
9,11,2012-01-01,72,48,60,7,54,56,5,0,724,1737,0,0,30.15,30.18,4.6,23,4.8


In [6]:
def process_weather():
    weather_file = pd.read_csv("Data/weather.csv", sep=',')
    weather_data = weather_file.drop('codesum', axis = 1) # codesum 컬럼 삭제
    
    data = weather_data.values[:,2:] # index, station_nbr, date를 제외한 나머지 데이터 array로 변경
    # 데이터가 있을 경우 float형으로 변경, 데이터가 없을 경우 unicodedata로 변경
    for col in range(data.shape[1]):
        for row in range(data.shape[0]):
            if not is_number(data[row, col]):
                if row == 0:
                    i = 1
                    while not is_number(data[row+i, col]):
                        i += 1
                    data[row, col] = data[row+i, col]
                else:
                    data[row, col] = data[row-1, col]
            data[row, col] = float(data[row, col]) # Convert string to float value

    weather_processed = np.append(weather_file.values[:,0:2], data, axis = 1)
    df = pd.DataFrame(weather_processed, index=None, columns=weather_data.columns.values)
    df.to_csv("Data/weather_processed.csv", sep=',', index=None)

In [7]:
#index를 제외한 key데이터를 dataframe로 가져옴
def match_store_station():
    key_file = pd.read_csv("Data/key.csv", sep=',')
    df = pd.DataFrame(key_file.values[:,1], index = key_file.values[:,0])
    return df

In [8]:
def read_processed_weather():
    weather_file = pd.read_csv("Data/weather_processed.csv", sep = ',')
    #date와 station_nbr정보를 가져옴
    multi_index = [weather_file.values[:,1], weather_file.values[:,0]]
    #index를 date와 station_nbr기준으로 하여 데이터를 가져옴
    df = pd.DataFrame(weather_file.values[:,2:], index=multi_index, columns=weather_file.columns.values[2:])
    return df

In [9]:
def get_weather_data(date, store_nbr, store_station_pairs, weather_file):
    station_nbr = store_station_pairs.loc[store_nbr].values[0]
    weather_data = weather_file.loc[date, station_nbr].values
    return weather_data

In [10]:
def get_item_list():
    #train data에서 item_nbr data중복 제거 후 list로 변환
    train_file = pd.read_csv("Data/train.csv", sep=',')
    item_list = list(set(list(train_file.values[:,2])))
    return item_list

In [11]:
def get_item_data(item_nbr, percent, store_station_pairs, weather_file, train_file):
    start = time.time()
    train_num = train_file.values[:,1].shape[0]

    item_index = np.where(train_file.values[:,2] == item_nbr)
    item_index = item_index[0]
    num = int(item_index.shape[0]*(1-percent)) # Select according to the given percentage, need to be improved
    iter_list = range(num, item_index.shape[0])
    item_dataset = np.zeros((len(iter_list), weather_file.values.shape[1]+1))
    count = 0
    #print len(iter_list)
    for i in iter_list:
        index = item_index[i]
        #print count
        date = train_file.values[:,0][index]
        store_nbr = train_file.values[:,1][index]
        units = train_file.values[:,3][index]

        weather_data = get_weather_data(date, store_nbr, store_station_pairs, weather_file)

        item_dataset[count,:] =  np.append(weather_data, [units], axis=0)
        count += 1
    end = time.time()
    #print "Running time for item %d: %f" %(int(item_nbr), (end-start))
    return item_dataset

In [12]:
def write_item_data(tasks):
    item_list = tasks.get()
    #print item_list
    store_station_pairs = match_store_station()
    weather_file = read_processed_weather() # index has 2 columns and values has 17 columns
    train_file = pd.read_csv("Data/train.csv", sep=',')

    #item_list = get_item_list()
    #item_nbr = item_list[0]
    for item_nbr in item_list:
        item_dataset = get_item_data(item_nbr, 0.1, store_station_pairs, weather_file, train_file) # Give percentage
        df = pd.DataFrame(item_dataset, index=None, columns=None)
        filename = "Item_data/item_%d.csv" %(int(item_nbr))
        df.to_csv(filename, sep=',', index=None, columns=None)

In [13]:
def multi_process(traget_func):
    processors = multiprocessing.cpu_count()

    myTasks = multiprocessing.Queue()
    item_list = get_item_list()
    temp_part = []
    div = len(item_list)/(processors-1)
    rem = len(item_list)%(processors-1)
    ind = 0
    while ind < div*(processors-1): 
        temp_part.append(item_list[ind:ind+div])
        ind = ind+div
    temp_part.append(item_list[(len(item_list)-rem):])

    for each in temp_part:
        myTasks.put(each)

    Workers = [multiprocessing.Process(target = traget_func, args =(myTasks,)) for i in range(processors)]

    #Workers[0].start()

    for each in Workers:
        each.start()

In [14]:
def read_item_data(item_nbr):
    filename = "Item_data/item_%d.csv" %(int(item_nbr))
    item_dataset = pd.read_csv(filename, sep=',')

    item_data = item_dataset.values[:,0:-1]
    item_label = item_dataset.values[:,-1]
    return item_data, item_label

In [15]:
def split_item_data(item_nbr, type):
    item_data, item_label = read_item_data(item_nbr)
    X_train, X_test, y_train, y_test = train_test_split(item_data, item_label, test_size=0.25, random_state=42)
    if type == 'train':
        return X_train, y_train
    elif type == 'test':
        return X_test, y_test

In [24]:
def get_full_data():
    train_file = pd.read_csv("Data/train.csv", sep=',')
    train_file = train_file[train_file['units'] > 0]

    store_station_pairs = match_store_station()
    #weather_file = read_weather() # index has 2 columns and values has 18 columns
    weather_file = read_processed_weather()
    train_num = train_file.values[:,1].shape[0]
    item_dataset = [] # Should be 20 columns
    for ind in tqdm(range(train_num)):
        store_nbr = train_file.values[:,1][ind]
        date = train_file.values[:,0][ind]        
        weather_data = get_weather_data(date, store_nbr, store_station_pairs, weather_file)
        #weather_data = np.append(train_file.values[:,2:][ind], weather_data, axis = 1) #[item_nbr, units, weather_features]
        weather_data = np.concatenate((train_file.values[:,2:][ind], weather_data), axis = 0)
        item_dataset.append(weather_data)

    item_dataset = np.array(item_dataset)
    #train_columns = np.append(np.array(['item_nbr','units']), weather_file.columns.values, axis=1)
    train_columns = np.concatenate((np.array(['item_nbr','units']), weather_file.columns.values), axis=0)
    train_df = pd.DataFrame(item_dataset, index=None, columns=train_columns) # Build item_nbr index on units and weather features
    train_df.to_csv("test_train_feature_matrix.csv", sep=',', index=None)
    return train_df

In [25]:
get_full_data()


  0%|          | 90/118696 [00:01<31:36, 62.55it/s]Exception in thread Thread-6:
Traceback (most recent call last):
  File "/Users/woongkook/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/woongkook/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/Users/woongkook/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 118696/118696 [30:58<00:00, 63.85it/s]


,item_nbr,units,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,9,29,52,31,42,16,36,40,23,0,716,1626,0,0.05,29.78,29.92,3.6,20,4.6
1,28,2,52,31,42,16,36,40,23,0,716,1626,0,0.05,29.78,29.92,3.6,20,4.6
2,51,1,52,31,42,16,36,40,23,0,716,1626,0,0.05,29.78,29.92,3.6,20,4.6
3,5,191,50,34,42,5,25,35,23,0,739,1729,0,0,29.13,30.52,11.4,32,11.3
4,44,215,50,34,42,5,25,35,23,0,739,1729,0,0,29.13,30.52,11.4,32,11.3
5,5,214,50,33,42,0,26,35,23,0,727,1742,0,0,29.15,30.54,10.3,32,10.2
6,45,112,50,33,42,0,26,35,23,0,727,1742,0,0,29.15,30.54,10.3,32,10.2
7,9,61,34,19,27,0,17,23,38,0,727,1742,0,0,29.34,30.09,22.8,30,22.5
8,27,21,34,19,27,0,17,23,38,0,727,1742,0,0,29.34,30.09,22.8,30,22.5
9,16,24,72,48,60,7,54,57,5,0,724,1737,0,0,30.15,30.19,4.4,25,4.8


In [26]:
def read_full_data():
    train_features = pd.read_csv("test_train_feature_matrix.csv", sep=',')
    #train_df = pd.DataFrame(train_features.values[:,1:], index=train_features.values[:,0], columns=train_features.columns.values[1:])
    train_df = pd.DataFrame(train_features.values, columns=train_features.columns)
    print(train_df)

In [27]:
train_features = pd.read_csv("test_train_feature_matrix.csv", sep=',')
#train_df = pd.DataFrame(train_features.values[:,1:], index=train_features.values[:,0], columns=train_features.columns.values[1:])
train_df = pd.DataFrame(train_features.values, columns=train_features.columns)
train_df

,item_nbr,units,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,9.0,29.0,52.0,31.0,42.0,16.0,36.0,40.0,23.0,0.0,716.0,1626.0,0.0,0.05,29.78,29.92,3.6,20.0,4.6
1,28.0,2.0,52.0,31.0,42.0,16.0,36.0,40.0,23.0,0.0,716.0,1626.0,0.0,0.05,29.78,29.92,3.6,20.0,4.6
2,51.0,1.0,52.0,31.0,42.0,16.0,36.0,40.0,23.0,0.0,716.0,1626.0,0.0,0.05,29.78,29.92,3.6,20.0,4.6
3,5.0,191.0,50.0,34.0,42.0,5.0,25.0,35.0,23.0,0.0,739.0,1729.0,0.0,0.00,29.13,30.52,11.4,32.0,11.3
4,44.0,215.0,50.0,34.0,42.0,5.0,25.0,35.0,23.0,0.0,739.0,1729.0,0.0,0.00,29.13,30.52,11.4,32.0,11.3
5,5.0,214.0,50.0,33.0,42.0,0.0,26.0,35.0,23.0,0.0,727.0,1742.0,0.0,0.00,29.15,30.54,10.3,32.0,10.2
6,45.0,112.0,50.0,33.0,42.0,0.0,26.0,35.0,23.0,0.0,727.0,1742.0,0.0,0.00,29.15,30.54,10.3,32.0,10.2
7,9.0,61.0,34.0,19.0,27.0,0.0,17.0,23.0,38.0,0.0,727.0,1742.0,0.0,0.00,29.34,30.09,22.8,30.0,22.5
8,27.0,21.0,34.0,19.0,27.0,0.0,17.0,23.0,38.0,0.0,727.0,1742.0,0.0,0.00,29.34,30.09,22.8,30.0,22.5
9,16.0,24.0,72.0,48.0,60.0,7.0,54.0,57.0,5.0,0.0,724.0,1737.0,0.0,0.00,30.15,30.19,4.4,25.0,4.8


In [29]:
train_file = pd.read_csv("Data/train.csv", sep=',')
train_file = train_file[train_file['units'] > 0]
train_file

,date,store_nbr,item_nbr,units
8,2012-01-01,1,9,29
27,2012-01-01,1,28,2
50,2012-01-01,1,51,1
115,2012-01-01,2,5,191
154,2012-01-01,2,44,215
226,2012-01-01,3,5,214
266,2012-01-01,3,45,112
341,2012-01-01,4,9,61
359,2012-01-01,4,27,21
459,2012-01-01,5,16,24


In [31]:
#result = pd.merge(train_file, train_df, on=['item_nbr','units'], how='outer') #16779086개 
result = pd.merge(train_file, train_df, on=['item_nbr','units'])
result

,date,store_nbr,item_nbr,units,tmax,tmin,tavg,depart,dewpoint,wetbulb,...,cool,sunrise,sunset,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,2012-01-01,1,9,29,52.0,31.0,42.0,16.0,36.0,40.0,...,0.0,716.0,1626.0,0.0,0.05,29.78,29.92,3.6,20.0,4.6
1,2012-01-01,1,9,29,45.0,22.0,34.0,15.0,10.0,28.0,...,0.0,718.0,1622.0,0.0,0.00,29.96,30.66,3.7,33.0,4.8
2,2012-01-01,1,9,29,28.0,10.0,19.0,-5.0,11.0,18.0,...,0.0,727.0,1743.0,0.0,0.00,29.26,30.12,8.9,16.0,9.2
3,2012-01-01,1,9,29,63.0,28.0,46.0,-8.0,25.0,36.0,...,0.0,718.0,1624.0,0.0,0.00,29.59,30.29,2.1,36.0,3.3
4,2012-01-01,1,9,29,27.0,8.0,18.0,-8.0,-1.0,14.0,...,0.0,718.0,1624.0,0.0,0.00,29.90,30.00,7.9,28.0,9.9
5,2012-01-01,1,9,29,50.0,30.0,40.0,8.0,27.0,32.0,...,0.0,721.0,1639.0,0.0,0.02,29.24,29.41,8.7,24.0,12.7
6,2012-01-01,1,9,29,57.0,34.0,46.0,9.0,24.0,36.0,...,0.0,736.0,1749.0,0.0,0.00,28.64,29.85,2.7,25.0,6.7
7,2012-01-01,1,9,29,49.0,29.0,39.0,1.0,22.0,31.0,...,0.0,733.0,1754.0,0.0,0.00,29.01,30.41,3.8,31.0,7.2
8,2012-01-01,1,9,29,52.0,25.0,39.0,9.0,11.0,28.0,...,0.0,718.0,1803.0,0.0,0.00,24.27,30.12,5.9,17.0,7.5
9,2012-01-01,1,9,29,48.0,29.0,39.0,10.0,21.0,32.0,...,0.0,640.0,1720.0,0.0,0.00,29.77,29.88,8.4,28.0,8.8
